In [ ]:
# 장중 정보기반 거래자의 비중과, 거래 상태를 HMM 으로 추론한다.
# Hidden 상태는 2개로 설정한다. 정보기반 거래자 및 유동성 거래자 상태 관측 데이터는
# 거래량 전날대비 1/500개 발생 때마다 매수 수량과 매도 수량을 측정하고, 매수 강도를 측정한다.
#
# 매수 강도 = 매수 수량 / 매도 수량
# 매수 강도가 1 에 가까워지면 정보보유 거래자의 참여 비중이 낮은 것이고,
# 1보다 커질수록 호재성 정보보유 거래자 비중이 높은것이고,
# 1보다 작아질수록 악재성 정보보유 거래자 비중이 높은 것임.
#
# 분석 데이터 : ETF Data


from hmmlearn import hmm
import numpy as np
import scipy.stats as stats
np.set_printoptions(precision=5)
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import os
from arch import arch_model
pd.options.mode.chained_assignment = None


def make_dfs(folder_path,ticker): #한 폴더에 있는 모든 파일 읽기
    file_names = sorted(os.listdir(folder_path))
    file_names = [file_name[8:14] for file_name in file_names]

    dfs = {}  # 데이터프레임을 저장할 딕셔너리

    for i, date in enumerate(file_names):
        df_name = f"df{i}"  # 변수 이름 생성
        df = read_data(date, ticker=ticker)  # 데이터 읽어오기
        dfs[df_name] = df  # 딕셔너리에 데이터프레임 저장
    return dfs
def make_dfs_pnl(folder_path,ticker): #한 폴더에 있는 모든 파일 읽기
    file_names = sorted(os.listdir(folder_path))
    file_names = [file_name[-10:-4] for file_name in file_names]
    dfs = {}  # 데이터프레임을 저장할 딕셔너리

    for i, date in enumerate(file_names):
        df_name = f"df{i}"  # 변수 이름 생성
        df = read_data_pnl(date, ticker=ticker)  # 데이터 읽어오기
        dfs[df_name] = df  # 딕셔너리에 데이터프레임 저장
    return dfs
def make_dfs_trade(folder_path,ticker): #한 폴더에 있는 모든 파일 읽기
    file_names = sorted(os.listdir(folder_path))
    file_names = [file_name[-10:-4] for file_name in file_names]
    dfs = {}  # 데이터프레임을 저장할 딕셔너리

    for i, date in enumerate(file_names):
        df_name = f"df{i}"  # 변수 이름 생성
        df = read_data_trade(date, ticker=ticker)  # 데이터 읽어오기
        dfs[df_name] = df  # 딕셔너리에 데이터프레임 저장
    return dfs
def read_data(date,ticker):
    df = pd.read_csv(f'C:/Users/HWPS/Desktop/data/trades/trades20{date}.csv')
    df = df[df['typeCode'] != 'UNKNOWN']
    df = df[df['isinCode'] == ticker]
    df = df[(df['timeStamp']>=90000000000) & (df['timeStamp']<=152000000000)]
    df = df.loc[:, ['timeStamp', 'price', 'volume', 'typeCode']]
    df['long_vol'] = df[df['typeCode'] == 'LONG']['volume']
    df['short_vol'] = df[df['typeCode'] == 'SHORT']['volume']
    df = df.reset_index(drop=True)
    df = df.fillna(0)
    df['timeStamp'] = '20'+ str(date)+ df['timeStamp'].astype(str).str.zfill(12)
    df['timeStamp'] = df['timeStamp'].apply(string_to_time)
    df['timeStamp'] = pd.to_datetime(df['timeStamp'])
    df = df[['timeStamp','price','volume','long_vol','short_vol']]
    return df
def read_data_pnl(date,ticker):
    df = pd.read_csv(f'C:/Users/HWPS/Desktop/data/pnl/pnl20{date}.csv')
    df = df[df['isinCode'] == ticker]
    df = df.loc[:, ['timestamp', 'cumRealizedPnL','tradeAmount','netAmount']]
    df = df.reset_index(drop=True)
    df['timestamp'] = '20'+ str(date)+ ' ' + df['timestamp'].str[:-3]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df[['timestamp','cumRealizedPnL','tradeAmount','netAmount']]
    df.rename(columns={'timestamp':'timeStamp'},inplace=True)
    return df
def read_data_trade(date,ticker):
    df = pd.read_csv(f'C:/Users/HWPS/Desktop/data/pnl/pnl20{date}.csv')
    df = df[df['isinCode'] == ticker]
    df = df.loc[:, ['timestamp', 'tradeAmount']]
    df = df.reset_index(drop=True)
    df['timestamp'] = '20'+ str(date)+ ' ' + df['timestamp'].str[:-3]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df[['timestamp','tradeAmount']]
    df.rename(columns={'timestamp':'timeStamp'},inplace=True)
    return df
def string_to_time(string_time):
    # 문자열을 시간, 분, 초로 분리
    year = int(string_time[:4])
    month = int(string_time[4:6])
    day = int(string_time[6:8])
    hours = int(string_time[8:10])
    minutes = int(string_time[10:12])
    seconds = int(string_time[12:14])
    microseconds = int(string_time[14:])
    # 시간 형식으로 변환
    time_data = datetime.strptime(f"{year:02d}-{month:02d}-{day:02d} {hours:02d}:{minutes:02d}:{seconds:02d}.{microseconds:06d}", "%Y-%m-%d %H:%M:%S.%f")
    # df[column_name] = df[column_name].astype(str).str.pad(12, side='left', fillchar='1') 다른 인덱싱
    return time_data.strftime("%Y-%m-%d %H:%M:%S.%f")
def cal_BVVPIN(df,df2,volume_slice):
    a = cal_TRVPIN(df,df2,volume_slice)
    a = a[['timeStamp', 'price']]
    a['diff_p'] = np.log(a['price']).diff()
    a = a.dropna()
    sigma = np.std(np.log(a['price']).diff())
    a['cdf_price'] = norm.cdf(a['diff_p']/sigma) #표준정규분포를 구하고,
    a['bvvpin'] = abs(a['cdf_price'] - (1-a['cdf_price']))
    return a
def all_file_bvvpin(dfs, volume_slice, rolling):
    try :
        dataframes = list(dfs.keys())
        bvvpin_results = pd.DataFrame()
        for i in range(len(dataframes)):
            if i == 0:
                bvvpin_results = bvvpin_results._append(
                    cal_BVVPIN(dfs[f'{dataframes[i]}'], dfs[f'{dataframes[i]}'], volume_slice))
            else:
                bvvpin_results = bvvpin_results._append(
                    cal_BVVPIN(dfs[f'{dataframes[i]}'], dfs[f'{dataframes[i - 1]}'], volume_slice))
    except :
        pass
    bvvpin_results['roll_bvvpin'] = bvvpin_results['bvvpin'].rolling(rolling).mean()
    bvvpin_results.reset_index(drop=True, inplace=True)
    return bvvpin_results
def all_file_trvpin(dfs, volume_slice, rolling):
    '''
    :param volume_slice: 전날 버킷을 몇개로 나눌건지
    :param rolling: rolling은 몇개로 할건지
    :return: trvpin_result
    '''
    dataframes = list(dfs.keys())
    trvpin_results = pd.DataFrame()
    for i in range(len(dataframes)):
        if i == 0:
            trvpin_results = trvpin_results._append(cal_TRVPIN(dfs[f'{dataframes[i]}'], dfs[f'{dataframes[i]}'], volume_slice))
        else:
            trvpin_results = trvpin_results._append(
                cal_TRVPIN(dfs[f'{dataframes[i]}'], dfs[f'{dataframes[i - 1]}'], volume_slice))

    trvpin_results['roll_trvpin'] = trvpin_results['trvpin'].rolling(rolling).mean()
    trvpin_results.reset_index(drop=True, inplace=True)
    return trvpin_results
def cal_TRVPIN(df,df2, volume_slice):
    volume = round(df2['volume'].sum()/(volume_slice),0)
    bv = 0  # buy volume
    sv = 0  # sell volume
    j = 0  # for VPIn
    sumv = 0
    leftover_bv = 0  # sell volume중 남은 것
    leftover_sv = 0  # buy volume 중 남은 것.
    cal_VPIN = pd.DataFrame(columns=['trvpin'])
    '''
    :param df:
    :param volume: Volume의 갯수를 구합니다.(몇개로 나눌지)
    :param roll_num: ROlling 할 갯수를 구합니다.
    :return: Tick Rule 기반의 VPIN의 값이 나옵니다.
    '''
    for i in range(len(df)):
        # bv나 sv 중 하나만 남으니까 leftover_bv or leftover_sv 로 나눈다.
        bv = bv + df['long_vol'].iloc[i] + leftover_bv
        sv = sv + df['short_vol'].iloc[i] + leftover_sv  # 다시 더해주기

        # leftover 초기화 해줘야함. 썼으니까.
        leftover_sv = 0  # leftover 초기화, 축적되면 안됨
        leftover_bv = 0  # leftover 초기화, 축적되면 안됨.

        # 바스켓을 만들어야 하니까. sumv를 계산해줌
        sumv = bv + sv  # sumv=bv+sv임.
        # sumv가 volume보다 클때 vpin을 만들어줌.

        while sumv >= volume:  # 바스켓이 하나 생긴 지점 #마지막 주문이 무엇인지 알아야함.
            cal_VPIN = cal_VPIN._append(df.iloc[[i], :])  # 여기서 나중에 시간만으로 바꿔줘야함.
            # 마지막 주문이 무엇인지.
            if df['short_vol'][i] != 0:  # 마지막 주문이 sell일때
                leftover_sv = sumv - volume  #
                cal_VPIN['trvpin'].iloc[j] = round(abs(bv - (volume - bv)) / volume,4)*100  # vpin쌓기 bv에
                bv = 0



            else :  # 마지막 주문이 buy
                leftover_bv = sumv - volume  #
                cal_VPIN['trvpin'].iloc[j] = round(abs(sv - (volume - sv)) / volume,4)*100  # vpin쌓기 bv에
                sv = 0
            sumv -= volume

            j += 1
            bv = 0
            sv = 0
        sumv = 0
    return cal_VPIN #d
def cal_TRVPIN2(df,volume_slice):
    volume = volume_slice
    bv = 0  # buy volume
    sv = 0  # sell volume
    j = 0  # for VPIn
    sumv = 0
    leftover_bv = 0  # sell volume중 남은 것
    leftover_sv = 0  # buy volume 중 남은 것.
    cal_VPIN = pd.DataFrame(columns=['trvpin'])
    '''
    :param df:
    :param volume: Volume의 갯수를 구합니다.(몇개로 나눌지)
    :param roll_num: ROlling 할 갯수를 구합니다.
    :return: Tick Rule 기반의 VPIN의 값이 나옵니다.
    '''
    for i in range(len(df)):
        # bv나 sv 중 하나만 남으니까 leftover_bv or leftover_sv 로 나눈다.
        bv = bv + df['long_vol'].iloc[i] + leftover_bv
        sv = sv + df['short_vol'].iloc[i] + leftover_sv  # 다시 더해주기

        # leftover 초기화 해줘야함. 썼으니까.
        leftover_sv = 0  # leftover 초기화, 축적되면 안됨
        leftover_bv = 0  # leftover 초기화, 축적되면 안됨.

        # 바스켓을 만들어야 하니까. sumv를 계산해줌
        sumv = bv + sv  # sumv=bv+sv임.
        # sumv가 volume보다 클때 vpin을 만들어줌.

        while sumv >= volume:  # 바스켓이 하나 생긴 지점 #마지막 주문이 무엇인지 알아야함.
            cal_VPIN = cal_VPIN._append(df.iloc[[i], :])  # 여기서 나중에 시간만으로 바꿔줘야함.
            # 마지막 주문이 무엇인지.
            if df['short_vol'][i] != 0:  # 마지막 주문이 sell일때
                leftover_sv = sumv - volume  #
                cal_VPIN['trvpin'].iloc[j] = round(abs(bv - (volume - bv)) / volume,4)*100  # vpin쌓기 bv에
                bv = 0



            else :  # 마지막 주문이 buy
                leftover_bv = sumv - volume  #
                cal_VPIN['trvpin'].iloc[j] = round(abs(sv - (volume - sv)) / volume,4)*100  # vpin쌓기 bv에
                sv = 0
            sumv -= volume

            j += 1
            bv = 0
            sv = 0
        sumv = 0
    return cal_VPIN #d##g#volume_
def draw_TR_VPIN(df):
    # DataFrame에서 필요한 데이터 선택
    data_to_plot = df[['timeStamp','roll_trvpin', 'price']]
    fig, ax1 = plt.subplots()
    # 첫 번째 y축을 설정
    ax1.plot(data_to_plot.index, data_to_plot['roll_trvpin'], 'b-')
    ax1.set_xlabel('Index')
    ax1.set_ylabel('roll_trvpin', color='b')
    # 두 번째 y축을 설정
    ax2 = ax1.twinx()
    ax2.plot(data_to_plot.index, data_to_plot['price'], 'r-')
    ax2.set_ylabel('price', color='r')
    plt.show()
def draw_BV_VPIN(df):
    # DataFrame에서 필요한 데이터 선택
    data_to_plot = df[['roll_bvvpin', 'price']]
    fig, ax1 = plt.subplots()
    # 첫 번째 y축을 설정
    ax1.plot(data_to_plot.index, data_to_plot['roll_bvvpin'], 'b-')
    ax1.set_xlabel('Index')
    ax1.set_ylabel('roll_bvvpin', color='b')
    # 두 번째 y축을 설정
    ax2 = ax1.twinx()
    ax2.plot(data_to_plot.index, data_to_plot['price'], 'r-')
    ax2.set_ylabel('price', color='r')
    plt.show()

def all_file_concat(dfs):
    dataframes = list(dfs.keys())
    results = pd.DataFrame()  # 빈 데이터프레임 생성
    for i in range(len(dataframes)):
        # 올바른 메소드인 append를 사용하여 데이터프레임을 연결
        results = results._append(dfs[f'{dataframes[i]}'])
    return results



#중기적 regime capture.
#return으로 HMM 돌려보기.
#주식시장의 변곡점 예측
bat_df = make_dfs('C:/Users/HWPS/Desktop/data/trades','KR7305720005')
bat_df['df1']['volume_sum_5min'] = bat_df['df1'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')
bat_df['df2']['volume_sum_5min'] = bat_df['df2'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')
bat_df['df3']['volume_sum_5min'] = bat_df['df3'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')
bat_df['df4']['volume_sum_5min'] = bat_df['df4'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')
bat_df['df5']['volume_sum_5min'] = bat_df['df5'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')
bat_df['df6']['volume_sum_5min'] = bat_df['df6'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')
bat_df['df18']['volume_sum_5min'] = bat_df['df18'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')

bat_df['df']['volume_sum_5min'] = bat_df['df6'].groupby(pd.Grouper(key='timeStamp', freq='10T'))['volume'].transform('sum')

bat_df['df18']

bat_df['df1']['volume_sum_5min'].unique()
bat_df['df2']['volume_sum_5min'].unique()
bat_df['df3']['volume_sum_5min'].unique()
resampled_df = bat_df.resample('5T')['volume'].sum()


def plot_bar_chart(data):
    x = np.arange(len(data))  # x 값으로 사용할 배열 생성
    plt.bar(x, data)
    plt.figure()  # 새로운 Figure 객체 생성
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('Bar Chart of Data')
    plt.show()


fig, axs = plt.subplots(2, 3, figsize=(15, 10))



# 레이아웃 조정
plt.tight_layout()
plt.show()
plot_bar_chart(bat_df['df1']['volume_sum_5min'].unique())
plot_bar_chart(bat_df['df2']['volume_sum_5min'].unique())
plot_bar_chart(bat_df['df3']['volume_sum_5min'].unique())
plot_bar_chart(bat_df['df4']['volume_sum_5min'].unique())
plot_bar_chart(bat_df['df5']['volume_sum_5min'].unique())
plot_bar_chart(bat_df['df6']['volume_sum_5min'].unique())
plot_bar_chart(bat_df['df18']['volume_sum_5min'].unique())

a = bat_df['df1']

b = bat_df['df2']
c = bat_df['df3']
d = bat_df['df4']
e = bat_df['df5']
f = bat_df['df6']
g = bat_df['df18']

a[a['volume']>999].value_counts('volume')
b[b['volume']>999].value_counts('volume')
c[c['volume']>999].value_counts('volume')
d[d['volume']>999].value_counts('volume')
e[e['volume']>999].value_counts('volume')
f[f['volume']>999].value_counts('volume')
g[g['volume']>3000].value_counts('volume')

b_twap = b[(b['volume']==1052)|(b['volume']==1051)]
g_twap = g[(g['volume']==3742)|(g['volume']==3741)]
def cal_vol_5min(dfs):
    for i, df in enumerate(dfs):
        # 날짜를 기준으로 데이터를 그룹화합니다.
        df_grouped = df.groupby(pd.Grouper(key='timeStamp', freq='D'))

        # 각 그룹 내에서 5분 간격으로 데이터를 합산합니다.
        summed_volume = df_grouped['volume'].resample('5Min').sum().reset_index(name='summed_volume')

        # 거래량을 합산한 데이터를 원본 데이터프레임에 병합합니다.
        merged_df = pd.merge(df, summed_volume, on='timeStamp', how='left')

        # NaN 값은 거래량이 없는 경우이므로 0으로 채웁니다.
        merged_df['summed_volume'] = merged_df['summed_volume'].fillna(0)

        # 새로운 열을 추가하여 합산된 거래량을 저장합니다.
        dfs[i] = merged_df
    return dfs

cal_vol_5min(bat_df)

bat_df['df1']['timeStamp']
bat_df2 = make_dfs('C:/Users/HWPS/Desktop/data/trades','KR7305540007')
bat_df_bvvpin_500 = all_file_bvvpin(bat_df,500,20)
bat_df2_bvvpin_100 = all_file_bvvpin(bat_df2,100,20)

bat_df2_bvvpin_500['diff_p'].value_counts()
aaaa = bat_df2_bvvpin_500[abs(bat_df2_bvvpin_500['diff_p'])>=50]
# 매수, 매도 수량의 비율을 계산한다
data = np.array(round(bat_df_bvvpin_500['bvvpin'],5))
data = np.reshape(data, [data.shape[0], 1])
nState = 3

# HMM 모델을 빌드한다
model = hmm.GaussianHMM(n_components=nState, tol=0.00001, n_iter=500000)
model.fit(data)

# 히든 상태를 추정한다
hState = model.predict(data)

np.bincount(hState)
bat_df_bvvpin_500['hstate'] = hState
bat_df_bvvpin_500['hstate'].value_counts()
# 추정된 히든 상태의 분포를 가져온다
mu = np.array(model.means_)
sigma = np.array([np.sqrt(x) for x in model.covars_])
sigma = np.reshape(sigma, [nState, 1])

# 추정된 히든 상태의 Transition 확률을 가져온다
P = np.array(model.transmat_)

# 결과를 확인한다
print("\nmu :\n", mu)
print("\nvol :\n", sigma)
print("\nTransition :\n", P)

# 히든 상태의 변화를 확인한다
plt.figure(figsize=(15, 6))
plt.plot(hState, 'r-')
plt.title("Change of hidden state", fontsize=15)
plt.grid(True, alpha=0.5)
plt.show()

# 매수 강도 차트를 그려본다
plt.figure(figsize=(15, 6))
plt.plot(data)
plt.title("Propotional of Buy/Sell", fontsize=15)
plt.axhline(y=1.0, color='red')
plt.grid(True, alpha=0.5)
plt.show()

dfa = bat_df2['df11']
bat_df2['df11'][bat_df2['df11']['volume']>400].value_counts('volume')
#롱숏이랑 별로 관련성이 없다.
asdf= bat_df2['df11'][bat_df2['df11']['volume']>=1000]


#state2 = high-vol 국면
# 히든 상태의 분포를 확인한다
plt.figure(figsize=(15, 6))
for i in range(nState):
    x = np.linspace(mu[i] - 4 * sigma[i], mu[i] + 4 * sigma[i], 100)
    plt.plot(x, stats.norm.pdf(x, mu[i], sigma[i]), linewidth=3, label='State : ' + str(i))
plt.title("Hidden States (" + str(nState) + " states)", fontsize=15)
plt.axvline(x=0.5, color='r', linestyle='--')
plt.legend(loc='upper left', fontsize=15)
plt.grid(True)
plt.show()

# 각 State의 비중을 계산한다. 이 비중으로 PIN을 추정할 수 있다.
print()
for i in range(nState):
    prob = len(np.where(hState == i)[0]) / len(hState)
    print("State %d : %.2f %s" % (i, prob * 100, '%'))



#data
def read_data(date,ticker):
    df = pd.read_csv(f'C:/Users/HWPS/Desktop/data/trades/trades20{date}.csv')
    df = df[df['typeCode'] != 'UNKNOWN']
    df = df[df['isinCode'] == ticker]
    df = df[(df['timeStamp']>=90000000000) & (df['timeStamp']<=152000000000)]
    df = df.loc[:, ['timeStamp', 'price', 'volume', 'typeCode']]
    df['long_vol'] = df[df['typeCode'] == 'LONG']['volume']
    df['short_vol'] = df[df['typeCode'] == 'SHORT']['volume']
    df = df.reset_index(drop=True)
    df = df.fillna(0)
    df['timeStamp'] = '20'+ str(date)+ df['timeStamp'].astype(str).str.zfill(12)
    df['timeStamp'] = df['timeStamp'].apply(string_to_time)
    df['timeStamp'] = pd.to_datetime(df['timeStamp'])
    df = df[['timeStamp','price','volume','long_vol','short_vol']]
    return df

def string_to_time(string_time):
    # 문자열을 시간, 분, 초로 분리
    year = int(string_time[:4])
    month = int(string_time[4:6])
    day = int(string_time[6:8])
    hours = int(string_time[8:10])
    minutes = int(string_time[10:12])
    seconds = int(string_time[12:14])
    microseconds = int(string_time[14:])
    # 시간 형식으로 변환
    time_data = datetime.strptime(f"{year:02d}-{month:02d}-{day:02d} {hours:02d}:{minutes:02d}:{seconds:02d}.{microseconds:06d}", "%Y-%m-%d %H:%M:%S.%f")
    # df[column_name] = df[column_name].astype(str).str.pad(12, side='left', fillchar='1') 다른 인덱싱
    return time_data.strftime("%Y-%m-%d %H:%M:%S.%f")
#df.info() # null 있는지 확인
#kodex 2차전지
df1 = read_data(240102,'KR7305720005')
df2 = read_data(240103,'KR7305720005')
df3 = read_data(240104,'KR7305720005')
df4 = read_data(240105,'KR7305720005')
df5 = read_data(240108,'KR7305720005')
df6 = read_data(240109,'KR7305720005')
df7 = read_data(240110,'KR7305720005')
df8 = read_data(240111,'KR7305720005')
df9 = read_data(240112,'KR7305720005')
df10 = read_data(240116,'KR7305720005')
##########################################################################################################################
##########################################################################################################################
##########################################################################################################################
'HMM_Informed / Regime Change'
#좀더 나이브하게 나눈 다음에~  아니면 amihud illiq에서 절대값 안쓰고 regime 써도 될듯.!


def cal_BS(df,df2,volume_slice):
    df = df.copy()
    volume = round(df2['volume'].sum()/volume_slice,0)
    bv = 0  # buy volume
    sv = 0  # sell volume
    j = 0  # for VPIn
    sumv = 0
    leftover_bv = 0  # sell volume중 남은 것
    leftover_sv = 0  # buy volume 중 남은 것.
    cal_BS = pd.DataFrame(columns=['B/V'])
    '''
    :param df:
    :param volume: Volume의 갯수를 구합니다.(몇개로 나눌지)
    :param roll_num: ROlling 할 갯수를 구합니다.
    :return: Tick Rule 기반의 VPIN의 값이 나옵니다.
    '''
    for i in range(len(df)):
        # bv나 sv 중 하나만 남으니까 leftover_bv or leftover_sv 로 나눈다.
        bv = bv + df['long_vol'].iloc[i] + leftover_bv
        sv = sv + df['short_vol'].iloc[i] + leftover_sv  # 다시 더해주기

        # leftover 초기화 해줘야함. 썼으니까.
        leftover_sv = 0  # leftover 초기화, 축적되면 안됨
        leftover_bv = 0  # leftover 초기화, 축적되면 안됨.

        # 바스켓을 만들어야 하니까. sumv를 계산해줌
        sumv = bv + sv  # sumv=bv+sv임.
        # sumv가 volume보다 클때 vpin을 만들어줌.

        while sumv >= volume:  # 바스켓이 하나 생긴 지점 #마지막 주문이 무엇인지 알아야함.
            cal_BS = cal_BS._append(df.iloc[[i], :])  # 여기서 나중에 시간만으로 바꿔줘야함.
            # 마지막 주문이 무엇인지.
            if df['short_vol'][i] != 0:  # 마지막 주문이 sell일때
                leftover_sv = sumv - volume  #
                cal_BS['B/V'].iloc[j] = round(bv/ volume,2)  # vpin쌓기 bv에
                bv = 0



            else :  # 마지막 주문이 buy
                leftover_bv = sumv - volume  #
                cal_BS['B/V'].iloc[j] = round((volume - sv)/volume,2)  # vpin쌓기 bv에
                sv = 0
            sumv -= volume

            j += 1
            bv = 0
            sv = 0
        sumv = 0
    return cal_BS


#slice를 50개.
df_BS_1 = cal_BS(df1,df1,50)
df_BS_2 = cal_BS(df2, df1,50)
df_BS_3 = cal_BS(df3,df2,50)
df_BS_4 = cal_BS(df4, df3,50)
df_BS_5 = cal_BS(df5,df4,50)

#slice를 200개
df_BS2_1 = cal_BS(df1,df1,200)
df_BS2_2 = cal_BS(df2, df1,200)
df_BS2_3 = cal_BS(df3,df2,200)
df_BS2_4 = cal_BS(df4, df3,200)
df_BS2_5 = cal_BS(df5,df4,200)


#slice를 500개
df1['volume'].sum()
df1_500 = cal_BS(df1,df1,500).reset_index(drop=True)
df2_500 = cal_BS(df2, df1,500).reset_index(drop=True)
df3_500 = cal_BS(df3,df2,500).reset_index(drop=True)
df4_500 = cal_BS(df4, df3,500).reset_index(drop=True)
df5_500 = cal_BS(df5,df4,500).reset_index(drop=True)
df6_500 = cal_BS(df6,df5,500).reset_index(drop=True)
df7_500 = cal_BS(df7, df6,500).reset_index(drop=True)
df8_500 = cal_BS(df8,df7,500).reset_index(drop=True)
df9_500 = cal_BS(df9, df8,500).reset_index(drop=True)
df10_500 = cal_BS(df10,df9,500).reset_index(drop=True)
df1_500['B/V_roll20'] = df1_500['B/V'].rolling(20).mean()

df1_500['B/V_exproll20'] = df1_500['B/V'].ewm(span=20, adjust=False).mean()
df2_500['B/V_exproll20'] = df2_500['B/V'].ewm(span=20, adjust=False).mean()


# B/V_roll20 열을 기반으로 선 그래프를 그립니다.
plt.plot(df1_500.dropna()['B/V_exproll20'], label='B/V_roll20', color='blue')

# 그래프 제목과 레이블 설정
plt.title('B/V_roll20 Chart')
plt.xlabel('Index')
plt.ylabel('B/V_roll20')

# 범례 추가
plt.legend()

# 그래프 표시
plt.show()
from statsmodels.tsa.stattools import adfuller
result = adfuller(df1_500['B/V_exproll20'])
result[1]


import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
# 1. 데이터 준비
data = df2_500['B/V_exproll20']

plot_acf(data, lags=10)  # lags는 최대 시차를 나타냅니다.
plt.title('Autocorrelation Function')
plt.xlabel('Lag')
plt.ylabel('ACF')
plt.show()

#딱히 관련이 없다.
df2_500[['B/V_exproll20','price']].corr()
df1_500['B/V']
np.mean(df5_500['B/V'])










df_BS_cc = pd.concat([df_BS_1,df_BS_2,df_BS_3,df_BS_4,df_BS_5])
df_BS_cc.reset_index(drop=True,inplace=True)
df_BS_cc_TR = pd.concat([df_BS_1,df_BS_2]) #트레이닝데이터
df_BS_cc_TR.reset_index(drop=True, inplace = True)#합친 데이터
df_BS_cc_TT = pd.concat([df_BS_3,df_BS_4,df_BS_5]) #테스트 데이터
#df_BS_cc.reset_index(drop=True, inplace = True)#합친 데이터



df_BS2_cc = pd.concat([df_BS2_1,df_BS2_2,df_BS2_3,df_BS2_4,df_BS2_5])
df_BS2_cc.reset_index(drop=True, inplace = True)#합친 데이터



# 매수, 매도 수량의 비율을 계산한다
data = np.array(df_BS2_cc['B/S'])
data = np.reshape(data, [data.shape[0], 1])

#상태 4개로 추정
nState = 3
# HMM 모델을 빌드한다
np.set_printoptions(precision=3)
model = hmm.GaussianHMM(n_components=nState, tol=0.00001, n_iter=5000)
model.fit(data)

# 히든 상태를 추정한다
hState = model.predict(data)
len(hState)
df_BS2_cc['Hidden'] = hState
# 추정된 히든 상태의 분포를 가져온다
mu = np.array(model.means_)
sigma = np.array([np.sqrt(x) for x in model.covars_])
sigma = np.reshape(sigma, [nState, 1])


# 추정된 히든 상태의 Transition 확률을 가져온다
P = np.array(model.transmat_)

# 결과를 확인한다
print("\nmu :\n", mu)
print("\nvol :\n", sigma)
print("\nTransition :\n", P)
ㅐ

# 히든 상태의 변화를 확인한다
plt.figure(figsize=(15, 6))
plt.plot(hState, 'r-')
plt.title("Change of hidden state", fontsize=15)
plt.grid(True, alpha=0.5)
plt.show()

# 매수 강도 차트를 그려본다
plt.figure(figsize=(15, 6))
plt.plot(data)
plt.title("Propotional of Buy/Sell", fontsize=15)
plt.axhline(y=1.0, color='red')
plt.grid(True, alpha=0.5)
plt.show()

# 히든 상태의 분포를 확인한다
plt.figure(figsize=(15, 6))
for i in range(nState):
    x = np.linspace(mu[i] - 4 * sigma[i], mu[i] + 4 * sigma[i], 100)
    plt.plot(x, stats.norm.pdf(x, mu[i], sigma[i]), linewidth=3, label='State : ' + str(i))
plt.title("Hidden States (" + str(nState) + " states)", fontsize=15)
plt.axvline(x=1, color='r', linestyle='--')
plt.legend(loc='upper left', fontsize=15)
plt.grid(True)
plt.show()

# 각 State의 비중을 계산한다. 이 비중으로 PIN을 추정할 수 있다.
print()
for i in range(nState):
    prob = len(np.where(hState == i)[0]) / len(hState)
    print("State %d : %.2f %s" % (i, prob * 100, '%'))
#상태 1 상태 3 상태 2는 뭘까?



###############################
'HMM으로 regime change 알아보기'#
###############################


##########################################################################################################################
##########################################################################################################################
##########################################################################################################################

#커버하는 ticker
'KR7091160002' #kodex 반도체 펀드
'KR7462330002' #kodex 2차전지산업레버리지
'KR7471990002' #kodex ai 반도체 핵심장비
'KR7305720005' #kodex 2차전지산업
'KR7462010000' #tiger 2차전지소재
'KR7102110004' #tiger 200 미래에셋
'KR7422420000' #kbstar 2차전지액티브
'KR7455850008' #sol 반도체소부장fn
'KR7396500001' #tiger fn반도체top10
'KR7364980003' #tiger 2차전지top10
'KR7461950008' #kodex 2차전지핵심소재
'KR7305540007' #tiger 2차전지테마
'KR7469070007' #KBSTAR AI&로봇
'KR7471760009' #Tiger AI반도체핵심공정
'KR7455860007' #sol 2차전지소부장
'KR7465330009' #KBSTAR 2차전지top10
'KR7388420002' #KBSTAR비메모리반도체액티브 펀드코드

#상태를 만들어보자.
dfs = make_dfs('C:/Users/HWPS/Desktop/data/trades','KR7471990002') #ai 반도체 핵심장비.


trvpin_50_50 = all_file_trvpin(dfs,50,50)
trvpin_100_50 = all_file_trvpin(dfs,100,50)
trvpin_500_50 = all_file_trvpin(dfs,500,50)

trvpin_100_50['diff_p'] = np.log(trvpin_100_50['price']).diff()
trvpin_500_50['diff_p'] = np.log(trvpin_500_50['price']).diff()

dfs['df11']


trvpin_100_50['price']

plt.plot(trvpin_100_50['price'])

# 그래프 제목 추가
plt.title('Price Plot')

# x축 레이블 추가
plt.xlabel('Index')

# y축 레이블 추가
plt.ylabel('Price')

# 그래프 표시
plt.show()
#test set
round(len(trvpin_100_50)*0.7)
round(len(trvpin_500_50)*0.7)

#regime make